In [49]:
import yaml
import os
import csv
import numpy as np
import pandas as pd
import glob
import json 
import matplotlib.pyplot as plt
from typing import Union, Tuple, List

In [50]:
config_path = '../../ted-sfc/config/smirk/pedestrian_crossing.yml'

In [51]:
with open(config_path, 'r') as _f:
    config = yaml.safe_load(_f)

In [52]:
grid_config = config["grid_config"]
detector_config = config["detector_config"]

# Grid variables
fps = grid_config["fps"]

# Detector variables
required_cell_subsets = detector_config["required_cell_subsets"]
calibration_videos = detector_config["detection_calibration_videos"]
cell_gap_time_limits = detector_config["cell_gap_time_limits"]
event_length_limit = detector_config["event_length_limit_seconds"]
cell_ranges = detector_config['attention']["cell_ranges"]

In [53]:
cell_ranges.items()

dict_items([(1, [2.7e-07, 5.5e-07]), (2, [6.5e-07, 1e-06]), (3, [1.44e-06, 1.84e-06]), (4, [3.1e-06, 3.5e-06]), (5, [6.3e-06, 6.8e-06]), (6, [1.28e-05, 1.34e-05])])

In [54]:
def get_matching_cell_key(morton_code: float, cell_ranges: dict) -> Union[int, None]:
    for cell, (min_value, max_value) in cell_ranges.items():
        if min_value <= morton_code <= max_value:
            return cell
    return None

In [55]:
morton_codes = pd.read_csv('../outputs/morton_codes_all_max_cell_only.csv')
morton_codes

,frame,morton
0,child/Y4kC5HWcultxGWAhuKdZw/cam000006.png,4296032516
1,child/XpIss6USqq1CvIRb2weqE/cam000280.png,67125252
2,child/BLt5th4KUYDe4tqqwQEbN/cam000128.png,4294967556
3,child/1zJO0PfH4rKy8MOl5x5di/cam000035.png,2251808403619848
4,child/0i0Pv5GElXKQI1OoIHfbY/cam000025.png,562952134467584
...,...,...
69296,child/FUTcU9qixnZnRtUKfR73U/cam000022.png,136315392
69297,child/Hg5IVmPBKY2WzuemE7p6J/cam000033.png,136314880
69298,child/4C874IC44CDuep4rdQFIN/cam000171.png,2251808405717512
69299,child/kJEr4oDRPCYEqv6dQgu6o/cam000043.png,1126174785814788


In [56]:
for i, row in morton_codes.iterrows():
    current_frame = row['frame']
    print(current_frame)
    print(int(row['morton']) / 10000000000)
    current_cell_key = get_matching_cell_key(int(row['morton']) / 10000000000, cell_ranges)

    break

child/Y4kC5HWcultxGWAhuKdZw/cam000006.png
0.4296032516


In [57]:
current_cell_key

In [58]:
smirk_child = '../datasets/smirk/child/'


In [59]:
mortons_seq = []
for seq in os.listdir(smirk_child):
    print(seq)
    if os.path.isdir(os.path.join(smirk_child, seq)): # reason for this is there is a labels.csv file in /child, need to skip it.
        for anno_frame in sorted(glob.glob(os.path.join(smirk_child, seq, '*.labels.png'))):
            #print(anno_frame)
            rgb_frame = anno_frame.replace('.labels.png', '.png').replace('../datasets/smirk/child', 'child')
            morton_frame = morton_codes.set_index('frame').loc[rgb_frame].values.astype(float) / 10000000000000
            current_cell_key = get_matching_cell_key(morton_frame, cell_ranges)
            print(morton_frame)
            print(current_cell_key)
            mortons_seq.append(list(morton_frame))
    break

Z8CrcBAx3TYsVgtucXJUV
[0.05497558]
None
[0.00087262]
None
[0.00087242]
None
[0.05497579]
None
[0.05497579]
None
[0.00087263]
None
[0.05583478]
None
[0.05497579]
None
[0.05497558]
None
[0.00087242]
None
[1.36314888e-05]
None
[0.00085899]
None
[0.0008592]
None
[1.34217736e-05]
None
[2.12992e-07]
None
[3.3288e-09]
None
[1.34217736e-05]
None
[2.13044e-07]
None
[2.130432e-07]
None
[1.34217736e-05]
None
[2.12992e-07]
None
[2.09716e-07]
None
[2.097664e-07]
None
[2.12992e-07]
None
[2.09716e-07]
None
[3.3288e-09]
None
[2.129928e-07]
None
[1.048832e-07]
None
[1.064964e-07]
None
[6.7109124e-06]
5
[1.064964e-07]
None
[6.8174084e-06]
None
[6.7108864e-06]
5
[6.710912e-06]
5
[6.8157444e-06]
None
[6.817408e-06]
None
[0.0004295]
None


In [60]:
class Sequence:
    def __init__(self, path, direction, interrupts=0):
        self.path = path
        self.interrupts = interrupts
        self.active = True
        self.direction = direction

In [87]:
seq = Sequence([1,1], 'left')
seq.path

[1, 1]

In [71]:
a: List[Sequence] = []
if not a:
    print('ss')

ss


In [156]:
path = [[1,1],[1,2],[2,3],[2,4],[2,5],[3,6],[4,7],[4,8],[5,9]]
# path = [[1,1],[1,2],[2,3],[2,4]]

In [166]:
def get_ordered_sequences(path: List[Tuple[int, int]], direction: str, tolerance=1):
    # TODO rename active_sequences
    active_sequences: List[Sequence] = []
    ordered_sequences: List[Sequence] = []
    # print(active_sequences)

    for node in path:
        # print(node)
        has_created_new_sequence = False

        if not active_sequences:
           
            seq = Sequence([node], direction)
            active_sequences.append(seq)
            print('I am here')
            for i in active_sequences:
                print(i.path)
            has_created_new_sequence = True
            continue

        active_sequences_copy = active_sequences.copy()

        for active_seq in active_sequences_copy:
            if not active_seq.active:
                continue

            prev = active_seq.path[-1]

            if node[0] > prev[0] and direction == "left":
                active_seq.path.append(node)
            elif node[0] < prev[0] and direction == "right":
                active_seq.path.append(node)
            elif node[0] == prev[0]:
                frame_diff = abs(node[1] - prev[1])

                if frame_diff > 1:   # frame_amount / fps
                    active_seq.active = False
                    ordered_sequences.append(active_seq)

                    seq = Sequence([node], direction)
                    active_sequences.append(seq)

                    # print(len(active_sequences))
                    has_created_new_sequence = True
            else:
                active_seq.interrupts += 1

                if active_seq.interrupts > tolerance:
                    active_seq.active = False
                    ordered_sequences.append(active_seq)

                if has_created_new_sequence:
                    continue

                seq = Sequence([node], direction)
                active_sequences.append(seq)
                has_created_new_sequence = True

    for active_seq in active_sequences:
        if len(active_seq.path) > 1 and active_seq.active:
            # print(len(active_seq.path))
            # print(len(active_sequences))
            # print(active_seq.active)
            ordered_sequences.append(active_seq)
            active_seq.active = False

    return ordered_sequences

In [167]:
ordered_sequences = get_ordered_sequences(path, 'left')
for i in ordered_sequences:
    print('herhe', i.path)

I am here
[[1, 1]]
herhe [[1, 1], [2, 3]]
herhe [[2, 5], [3, 6], [4, 7], [5, 9]]


In [173]:
required_cell_subsets = config['detector_config']['required_cell_subsets']
required_cell_subsets

[{1, 2, 3}, {4, 5, 6}]

In [170]:
sequence_path = [[2, 5], [3, 6], [4, 7], [5, 9]]

In [171]:
unique_cells = {s[0] for s in sequence_path}
unique_cells

{2, 3, 4, 5}

In [176]:
for required_cell_subset in required_cell_subsets:
    count = len(required_cell_subset & unique_cells)
    print(count)

2
2


In [28]:
count = 0
active_path = []
ordered_path = []
direction = 'left'
for node in path:
    seq = Sequence([node], direction)
    active_path.append(seq)    
    print('start',len(active_path))
    
    for i in active_path:
        
        if not i.active:
            continue
            
        prev = i.path[-1]
        print('prev',prev)
        print('node0', node[0])
        print('prev0',prev[0])
        if node[0] == prev[0]:
             frame_diff = abs(node[1] - prev[1])

             if frame_diff / 10 > 3:
                i.active = False
                ordered_path.append(i)
                seq = Sequence([node], direction)
                active_sequences.append(seq)
                has_created_new_sequence = True
           
        if node[0] > prev[0] and direction == "left":
            i.path.append(node)
        elif node[0] < prev[0] and direction == "right":
            i.path.append(node)
        else:
            i.interrupts += 1

            if i.interrupts > 5: # 5 is the tolarence
                i.active = False 
                ordered_path.append(i)
        
        print('frame_diff:', frame_diff)
        print('iiii:',i.path)
    count+=1

    if count == 9:
        break
    

start 1
prev [1, 1]
node0 1
prev0 1
frame_diff: 0
iiii: [[1, 1]]
start 2
prev [1, 1]
node0 1
prev0 1
frame_diff: 1
iiii: [[1, 1]]
prev [1, 2]
node0 1
prev0 1
frame_diff: 0
iiii: [[1, 2]]
start 3
prev [1, 1]
node0 2
prev0 1
frame_diff: 0
iiii: [[1, 1], [2, 3]]
prev [1, 2]
node0 2
prev0 1
frame_diff: 0
iiii: [[1, 2], [2, 3]]
prev [2, 3]
node0 2
prev0 2
frame_diff: 0
iiii: [[2, 3]]
start 4
prev [2, 3]
node0 2
prev0 2
frame_diff: 1
iiii: [[1, 1], [2, 3]]
prev [2, 3]
node0 2
prev0 2
frame_diff: 1
iiii: [[1, 2], [2, 3]]
prev [2, 3]
node0 2
prev0 2
frame_diff: 1
iiii: [[2, 3]]
prev [2, 4]
node0 2
prev0 2
frame_diff: 0
iiii: [[2, 4]]
start 5
prev [2, 3]
node0 2
prev0 2
frame_diff: 2
iiii: [[1, 1], [2, 3]]
prev [2, 3]
node0 2
prev0 2
frame_diff: 2
iiii: [[1, 2], [2, 3]]
prev [2, 3]
node0 2
prev0 2
frame_diff: 2
iiii: [[2, 3]]
prev [2, 4]
node0 2
prev0 2
frame_diff: 1
iiii: [[2, 4]]
prev [2, 5]
node0 2
prev0 2
frame_diff: 0
iiii: [[2, 5]]
start 6
prev [2, 3]
node0 3
prev0 2
frame_diff: 0
iiii: [

In [88]:
i


In [81]:
for i in a:
    prev = i.path[-1]
    print(prev)

[1, 1]
[2, 2]
[2, 3]
[2, 4]
[2, 5]
[2, 6]
[2, 7]
[2, 8]
[2, 9]


In [23]:
a = False
if not a:
    print('ss')

ss
